## Demonstration of Name-Based Access Control and Group Keys

Uses Python scripts on FABRIC nodes to exchange files and perform key exchanges based upon NDN's NAC designs.

While it does not run on an active NDN-DPDK node like intended in FABRIC_NAC, it demonstrates the key exchange in accordance with the separation of producer and data owner nodes.

See attached report for image sources.

### Deallocate Slice

This content is placed at the top to avoid mistakes from people running through the notebook using Shift-Enter.
Note that `fablib` is not imported unless the first code block after this section is run. 

In [ ]:
slice_name="NACgroupKey-" + fablib.get_bastion_username()
slice = fablib.get_slice(name=slice_name)
fablib.delete_slice(slice_name)

### Initial Setup (from fabric-ndn)

The bastion key and slice key must be in your `fabric_config/` JuPyter directory in order to properly operate the FabLib library on FABRIC.

Try to avoid skipping steps to prevent missing variables/imports.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager()
conf = fablib.show_config()

### Topology Configuration (run to load variables)

In [ ]:
#at current the topology is intended to allocate on a single site

#configuring nodes for topology in *_conf variables
node_conf = [
    {"name": "dataowner", "cores": 2, "ram": 8, "disk": 20, "image": 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils', 'pwntools']},
    {"name": "producer",  "cores": 2, "ram": 8, "disk": 20, "image": 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils', 'pwntools']},
    {"name": "consumer",  "cores": 2, "ram": 8, "disk": 20, "image": 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils', 'pwntools']},
    #{"name": "attacker",  "cores": 2, "ram": 8, "disk": 20, "image": 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils', 'pwntools']},
    {"name": "router1",   "cores": 2, "ram": 8, "disk": 20, "image": 'default_ubuntu_20', 'packages': ['net-tools', 'iperf3', 'moreutils', 'pwntools']}
]

net_conf = [
    {"name": "net_source", "subnet": "10.10.1.0/24", "nodes": [
        {"name": "dataowner", "addr": "10.10.1.30"},
        {"name": "producer", "addr": "10.10.1.40"},
        {"name": "router1",  "addr": "10.10.1.10"}
    ]},
    {"name": "net_user", "subnet": "10.10.2.0/24", "nodes": [
        {"name": "consumer",  "addr": "10.10.2.50"},
        #{"name": "attacker",  "addr": "10.10.2.60"},
        {"name": "router1",   "addr": "10.10.2.10"}
    ]}
]

route_conf = [
    {"addr": "10.10.1.0/24", "gw": "10.10.1.10", "nodes": ["dataowner", "producer"]},
    {"addr": "10.10.2.0/24", "gw": "10.10.2.10", "nodes": ["consumer"]} #once implemented, potentially include    , "attacker"
]

#calculate config values for use in determining a valid site for allocation
exp_conf = {'cores': sum([ n['cores'] for n in node_conf]), 'nic': sum([len(n['nodes']) for n in net_conf]), 'ram': sum([ n['ram'] for n in node_conf]) }

print("Configuration setup loaded.")

### Slice existence check and allocation

In [ ]:
slice_name="NACgroupKey-" + fablib.get_bastion_username()
try:
    slice = fablib.get_slice(name=slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

In [ ]:
## Run if Slice is not allocated

while True:
    site_name = fablib.get_random_site()
    #check if randomly chosen site has the resources to properly allocate
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_conf['cores']) and
        (fablib.resources.get_ram_available(site_name) > 1.2*exp_conf['ram']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_conf['nic']) ):
        break

for n in node_conf:
    slice.add_node(name=n['name'], site=site_name, 
                   cores=n['cores'], 
                   ram=n['ram'], 
                   disk=n['disk'], 
                   image=n['image'])
for n in net_conf:
    ifaces = [slice.get_node(node["name"]).add_component(model="NIC_Basic", 
                                                 name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

fablib.show_site(site_name)

In [ ]:
#wait for visible output from previous
#it may be helpful to spot-check the resources availble on the other node
slice.submit()

#wait for allocation success
slice.get_state()
slice.wait_ssh(progress=True)

#NOTE: examine output of this and the above block carefully to ensure the slice has been properly built 

### Configure resources within nodes

Step 1: Update Nodes, Install Useful Packages, Load Scripts

In [ ]:
for n in node_conf:
    if len(n['packages']):
        node = slice.get_node(n['name'])
        pkg = " ".join(n['packages'])
        node.execute_thread("sudo apt update; sudo apt -y install %s" % pkg)

from ipaddress import ip_address, IPv4Address, IPv4Network
for net in net_conf:
    for n in net['nodes']:
        if_name = n['name'] + '-' + net['name'] + '-p1'
        iface = slice.get_interface(if_name)
        iface.ip_link_up()
        if n['addr']:
            iface.ip_addr_add(addr=n['addr'], subnet=IPv4Network(net['subnet']))
        else:
            iface.get_node().execute("sudo ip addr flush dev %s"  % iface.get_device_name())

hosts_txt = [ "%s\t%s" % ( n['addr'], n['name'] ) for net in net_conf  for n in net['nodes'] if type(n) is dict and n['addr']]

for n in slice.get_nodes():
    for h in hosts_txt:
        n.execute("echo %s | sudo tee -a /etc/hosts" % h)

for n in slice.get_nodes():
    n.execute("sudo sysctl -w net.ipv4.ip_forward=1")

for rt in route_conf:
    for n in rt['nodes']:
        slice.get_node(name=n).ip_route_add(subnet=IPv4Network(rt['addr']), gateway=rt['gw'])

In [ ]:
print("Place scripts in environments.")
nodes = slice.get_nodes()
for node in nodes:
    try:
        #Python scripts for operating
        result = node.upload_file('demoScripts/consumer.py','consumer.py')
        result = node.upload_file('demoScripts/producer.py','producer.py')
        result = node.upload_file('demoScripts/datowner.py','datowner.py')
        result = node.upload_file('demoScripts/ndnroute.py','ndnroute.py')
    except Exception as e:
        print("Failed on node %s for the following reason:"%(node.get_name()))
        print(e.message)


Step 2: Instructions to perform demo